In [1]:
import xml.etree.ElementTree as ET
import xml_output_parser as xo
from importlib import reload

## Tests

In [67]:
defxml = "outputs/test_default_3-C-FIX-NOZ-.152987-20250729.xml"
stdxml = "outputs/test_standard_3-C-FIX-NOZ-.152990-20250729.xml"
stdpass = 'test_outs/std_PASS.xml'

In [70]:
xo.get_xmlener(defxml, enertype='correlation energy', verbose=True)

MP2-F12 correlation energy: -0.306939237510969


-0.306939237510969

In [71]:
xo.get_xmlener(stdxml, verbose=True)

MP2-F12 total energy: -128.803304073533


-128.803304073533

In [74]:
reload(xo)
xo.get_xmlener(stdpass, enertype='total energy', command='MP2-F12', verbose=True)

ERROR READING ENERGY =============================
test_outs/std_PASS.xml does not contain a single unique 'MP2-F12' method jobstep


ValueError: not enough values to unpack (expected 1, got 0)

# Inspecting tree
This section is for viewing the xml tree visually and getting the important tags and attribute keys that would be used in the actual value retrieval code above.

In [7]:
def print_attrib(attrib):
    for key, val in attrib.items():
        str_to_print = f"{key:20s}"
        try:
            str_to_print += f" {val:20.15f}"
        except:
            str_to_print += f" {val:20s}"
        print(str_to_print)
        
def iter_inspect(node, space='', tag=None, verbose=False):
    """
    Visually inspect the tree, iterative through all sub-children,
    starting with `node` as root.
    """
    
    d = 0
    for child in node:
        s = space + f" {d} "
        if tag is None or tag == child.tag:
            print(f"{s} {child.tag}")
            if verbose:
                print_attrib(child.attrib)
        
        iter_inspect(child, space=s, tag=tag, verbose=verbose)
        d += 1

In [8]:
tree = xo.get_clean_tree(stdpass)
root = tree.getroot()

In [9]:
# Print all without verbose
iter_inspect(root)

 0  job
 0  0  metadataList
 0  0  0  metadata
 0  0  1  metadata
 0  0  2  metadata
 0  1  platform
 0  1  0  version
 0  1  0  0  date
 0  1  1  licence
 0  1  2  parallel
 0  1  3  dimensions
 0  2  jobstep
 0  2  0  molecule
 0  2  0  0  symmetry
 0  2  0  1  atomArray
 0  2  0  1  0  atom
 0  2  0  1  1  atom
 0  2  0  2  bondArray
 0  2  0  2  0  bond
 0  2  1  property
 0  2  2  property
 0  2  3  property
 0  2  4  property
 0  2  5  property
 0  2  6  property
 0  2  7  property
 0  2  8  time
 0  2  9  storage
 0  2  10  summary
 0  3  jobstep
 0  3  0  property
 0  3  1  property
 0  3  2  property
 0  3  3  property
 0  3  4  property
 0  3  5  time
 0  3  6  storage
 0  3  7  summary
 0  4  jobstep
 0  4  0  property
 0  4  1  property
 0  4  2  property
 0  4  3  property
 0  4  4  property
 0  4  5  property
 0  4  6  property
 0  4  7  property
 0  4  8  property
 0  4  9  property
 0  4  10  time
 0  4  11  storage
 0  4  12  summary
 0  5  input
 0  5  0  p
 0  5  1  

In [11]:
# Print jobstep w/o verbose
iter_inspect(root, tag='jobstep', verbose=True)

 0  2  jobstep
command              DF-HF-SCF           
commandset           SCFPRO              
 0  3  jobstep
command              DF-MP2              
commandset           CCSD                
 0  4  jobstep
command              DF-MP2-F12          
commandset           CCSD                


In [17]:

# Print a jobstep w verbose
final_job = root[0][4]
iter_inspect(final_job,verbose=True)

 0  property
name                 energy              
method               Reference           
stateSymmetry        1                   
stateNumber          1                   
stateMultiplicity    1                   
value                -621.438346039003   
 1  property
name                 OVERLAP             
method               F12 singles correction
stateSymmetry        1                   
stateNumber          1                   
stateMultiplicity    1                   
value                -0.617019861676397E-02
 2  property
name                 energy              
method               New reference       
stateSymmetry        1                   
stateNumber          1                   
stateMultiplicity    1                   
value                -621.444516237619   
 3  property
name                 singlet correction  
method               F12/3C(F)           
stateSymmetry        1                   
stateNumber          1                   
stateMultiplicity   

In [17]:
print_attrib(final_job[9])

name                 total energy        
method               MP2-F12/3C(F)       
principal            true                
stateSymmetry        1                   
stateNumber          1                   
stateMultiplicity    1                   
value                -621.941030126874   


In [ ]:
final_job[9].items()

In [ ]:

mp2f12_elems = root.findall(".//property/[@method='MP2-F12/3C(F)']")
iter_inspect(mp2f12_elems, verbose=True)



In [66]:
reload(xo)
display(xo.get_xmlener(stdpass, verbose=True))
display(xo.get_xmlener(stdpass, enertype='correlation energy', verbose=True))
display(xo.get_xmlener(stdpass, command='DF-MP2', verbose=True))
display(xo.get_xmlener(stdpass, command='DF-MP2', enertype='correlation energy', verbose=True))

MP2-F12/3C(F) total energy: -621.941030126874


-621.941030126874

MP2-F12/3C(F) correlation energy: -0.496513889254892


-0.496513889254892

MP2 total energy: -621.596134003812


-621.596134003812

MP2 correlation energy: -0.157787964809859


-0.157787964809859

In [23]:
print_attrib(final_job)

command              DF-MP2-F12          
commandset           CCSD                


In [61]:
xo.find_by_attrib([final_job], 'command', 'DF-MP2')

[]

In [31]:
for js in root.findall('.//jobstep'):
    print_attrib(js)

command              DF-HF-SCF           
commandset           SCFPRO              
command              DF-MP2              
commandset           CCSD                
command              DF-MP2-F12          
commandset           CCSD                


In [63]:
import re
reload(xo)
dfmp2, = xo.find_by_attrib(root.findall('.//jobstep'), 'command', "DF-MP2")
dfmp2f12, = xo.find_by_attrib(root.findall('.//jobstep'), 'command', 'DF-MP2-F12')
print(dfmp2, dfmp2f12)

<Element 'jobstep' at 0x14fafcf68090> <Element 'jobstep' at 0x14fafcf68360>


In [64]:
print_attrib(dfmp2)
print_attrib(dfmp2f12)

command              DF-MP2              
commandset           CCSD                
command              DF-MP2-F12          
commandset           CCSD                


In [57]:
iter_inspect(dfmp2, verbose=True)

 0  property
name                 energy              
method               Reference           
stateSymmetry        1                   
stateNumber          1                   
stateMultiplicity    1                   
value                -621.438346039003   
 1  property
name                 singlet pair energy 
method               MP2                 
stateSymmetry        1                   
stateNumber          1                   
stateMultiplicity    1                   
value                -0.93537971033219E-01
 2  property
name                 triplet pair energy 
method               MP2                 
stateSymmetry        1                   
stateNumber          1                   
stateMultiplicity    1                   
value                -0.642499937766403E-01
 3  property
name                 correlation energy  
method               MP2                 
stateSymmetry        1                   
stateNumber          1                   
stateMultiplicity    

In [62]:
xo.find_by_attrib(dfmp2, 'name', 'correlation energy')

[<Element 'property' at 0x14fafcf681d0>]

In [75]:
import tabulate_outs as to